# Databricks Lakebase Inventory App - Automated Deployment

This notebook automates the deployment of the Lakebase Inventory Management System.

## What This Notebook Does:
1. Creates a Lakebase (Managed Postgres) database instance
2. Creates a Databricks App
3. Creates a Secret Scope with Flask secret key
4. Adds the Lakebase database as an app resource
5. Clones the GitHub repository to the workspace
6. Creates an app.yaml configuration file
7. Deploys the Databricks App
8. Provides redeployment commands

## Prerequisites:
- Unity Catalog enabled workspace
- Appropriate permissions to create databases, apps, and secret scopes
- Databricks SDK installed


## Step 0: Install Required Libraries


In [ ]:
%pip install databricks-sdk --upgrade
dbutils.library.restartPython()


## Configuration Section

**IMPORTANT**: Update these configuration values before running the notebook!


In [ ]:
# ============= CONFIGURATION - EDIT THESE VALUES =============

# Database Configuration
LAKEBASE_DB_NAME = "inventory-lakebase-db"  # Name for your Lakebase database (DNS compliant: alphanumeric and hyphens only)
LAKEBASE_CAPACITY = "CU_2"  # Capacity: CU_1, CU_2, CU_4, or CU_8 (higher = more compute power)

# App Configuration
APP_NAME = "inventory-management-app"  # Name for your Databricks App
APP_DESCRIPTION = "Databricks Lakebase Inventory Management System"

# Secret Scope Configuration
SECRET_SCOPE_NAME = "app-secrets"  # Name for your secret scope
SECRET_KEY_NAME = "SECRET_KEY"  # Key name for Flask secret

# Repository Configuration
REPO_URL = "https://github.com/reynoldspravindev/databricks-lakebase-inventory"
REPO_PATH = "/Repos/Production/databricks-lakebase-inventory"  # Workspace path for repo
REPO_PROVIDER = "github"  # git provider

# Postgres Schema and Table Names
PGDATABASE = "databricks_postgres"
POSTGRES_SCHEMA = "inventory_app"
POSTGRES_TABLE = "inventory_items"
POSTGRES_CATEGORY_TABLE = "inventory_category"
POSTGRES_WAREHOUSE_TABLE = "inventory_warehouse"
POSTGRES_SUPPLIER_TABLE = "inventory_supplier"
POSTGRES_DEMAND_TABLE = "inventory_demand_forecast"

# Data Management
FORCE_DATA_RESET = "false"  # Set to "true" to reset data on startup
LOAD_SAMPLE_DATA = "true"   # Set to "false" to disable sample data loading

# Dashboard Configuration (optional - update after creating dashboard)
DASHBOARD_ID = ""  # Leave empty for now, update later if needed

# ==============================================================


## Initialize Databricks Workspace Client


In [ ]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import apps, compute, workspace as ws_service
from databricks.sdk.service.apps import AppResource
import secrets
import time
import json
from datetime import timedelta

# Initialize workspace client using notebook-native authentication
w = WorkspaceClient()

# Get current workspace info
current_user = w.current_user.me()

# Get the correct workspace URL from the Databricks context
# In a notebook, we can get the workspace URL from the notebook context
try:
    # Try to get workspace URL from dbutils (more reliable in notebooks)
    workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
    workspace_url = f"https://{workspace_url}"
except:
    # Fallback to config.host if dbutils is not available
    workspace_url = w.config.host

print(f"✓ Connected to workspace: {workspace_url}")
print(f"✓ Current user: {current_user.user_name}")
print(f"✓ User ID: {current_user.id}")


## Step 1: Create Lakebase (Managed Postgres) Database Instance

This creates a managed PostgreSQL database instance in Unity Catalog.


In [ ]:
print("Step 1: Creating Lakebase Database Instance...\n")

try:
    # Create the Lakebase database instance
    # Using the correct SDK method: create_database_instance_and_wait
    from databricks.sdk.service.database import DatabaseInstance
    
    database_instance = DatabaseInstance(
        name=LAKEBASE_DB_NAME,
        capacity=LAKEBASE_CAPACITY  # Valid values: "CU_1", "CU_2", "CU_4", "CU_8"
    )
    
    print(f"Creating database '{LAKEBASE_DB_NAME}' with capacity: {LAKEBASE_CAPACITY}")
    
    # Create and wait for the database instance to be ready
    lakebase_db = w.database.create_database_instance_and_wait(
        database_instance=database_instance,
        timeout=timedelta(minutes=20)
    )
    
    print(f"✓ Lakebase database '{LAKEBASE_DB_NAME}' created successfully!")
    print(f"  Database Name: {lakebase_db.name}")
    print(f"  Status: {lakebase_db.state}")
    
    # Store database details
    LAKEBASE_DB_ID = lakebase_db.name
    PGDATABASE = lakebase_db.name
    
except Exception as e:
    # Check if database already exists
    if "already exists" in str(e).lower() or "AlreadyExists" in str(e):
        print(f"⚠ Database '{LAKEBASE_DB_NAME}' already exists. Using existing database.")
        # List databases and find ours
        databases = list(w.database.list_database_instances())
        lakebase_db = next((db for db in databases if db.name == LAKEBASE_DB_NAME), None)
        if lakebase_db:
            LAKEBASE_DB_ID = lakebase_db.name
            PGDATABASE = lakebase_db.name
            print(f"  Database Name: {LAKEBASE_DB_ID}")
            print(f"  Status: {lakebase_db.state}")
        else:
            raise Exception(f"Could not find database '{LAKEBASE_DB_NAME}'")
    else:
        print(f"✗ Error creating Lakebase database: {e}")
        raise

print(f"\n" + "="*60)


## Step 2: Generate Flask Secret Key


In [ ]:
print("Step 2: Generating Flask Secret Key...\n")

# Generate a secure random secret key for Flask
FLASK_SECRET_KEY = secrets.token_hex(32)

print(f"✓ Flask secret key generated successfully!")
print(f"  Key length: {len(FLASK_SECRET_KEY)} characters")
print(f"  Preview: {FLASK_SECRET_KEY[:16]}...{FLASK_SECRET_KEY[-16:]}")

print(f"\n" + "="*60)


## Step 3: Create Secret Scope and Add Flask Secret Key


In [ ]:
print("Step 3: Creating Secret Scope...\n")

try:
    # Create secret scope
    w.secrets.create_scope(scope=SECRET_SCOPE_NAME)
    print(f"✓ Secret scope '{SECRET_SCOPE_NAME}' created successfully!")
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"⚠ Secret scope '{SECRET_SCOPE_NAME}' already exists. Using existing scope.")
    else:
        print(f"✗ Error creating secret scope: {e}")
        raise

# Add Flask secret key to the scope
try:
    w.secrets.put_secret(
        scope=SECRET_SCOPE_NAME,
        key=SECRET_KEY_NAME,
        string_value=FLASK_SECRET_KEY
    )
    print(f"✓ Secret key '{SECRET_KEY_NAME}' added to scope successfully!")
except Exception as e:
    print(f"✗ Error adding secret: {e}")
    raise

# Verify the secret was created
secrets_list = list(w.secrets.list_secrets(scope=SECRET_SCOPE_NAME))
print(f"\nSecrets in scope '{SECRET_SCOPE_NAME}':")
for secret in secrets_list:
    print(f"  - {secret.key}")

print(f"\n" + "="*60)


## Step 4: Clone GitHub Repository to Workspace


In [ ]:
print("Step 4: Cloning GitHub Repository...\n")

try:
    # Create the repo
    repo = w.repos.create(
        url=REPO_URL,
        provider=REPO_PROVIDER,
        path=REPO_PATH
    )
    
    print(f"✓ Repository cloned successfully!")
    print(f"  Repository ID: {repo.id}")
    print(f"  Repository Path: {repo.path}")
    print(f"  Branch: {repo.branch}")
    print(f"  URL: {repo.url}")
    
    REPO_WORKSPACE_PATH = repo.path
    
except Exception as e:
    if "already exists" in str(e).lower():
        print(f"⚠ Repository already exists at '{REPO_PATH}'. Using existing repository.")
        # Get the existing repo
        try:
            repo = w.repos.get(repo_id=REPO_PATH)
            REPO_WORKSPACE_PATH = repo.path
            print(f"  Repository Path: {REPO_WORKSPACE_PATH}")
        except:
            REPO_WORKSPACE_PATH = REPO_PATH
            print(f"  Using path: {REPO_WORKSPACE_PATH}")
    else:
        print(f"✗ Error cloning repository: {e}")
        raise

print(f"\n" + "="*60)


## Step 5: Create app.yaml Configuration File


In [ ]:
print("Step 5: Creating app.yaml Configuration File...\n")

# Create app.yaml content with resources configuration
app_yaml_content = f"""command: ["python", "app.py"]

# App resources - database and secret scope
resources:
  - name: database
    database:
      name: {LAKEBASE_DB_NAME}
      permission: CAN_CONNECT
  - name: secrets
    secret_scope:
      scope: {SECRET_SCOPE_NAME}
      permission: CAN_READ

env:
  - name: 'DATABRICKS_HOST'
    value: '{workspace_url}'
  - name: 'PGDATABASE'
    value: '{PGDATABASE}'
  - name: 'POSTGRES_SCHEMA'
    value: '{POSTGRES_SCHEMA}'
  - name: 'PGPORT'
    value: "5432"
  - name: 'PGSSLMODE'
    value: "require"
  - name: 'PGAPPNAME'
    value: "inventory_app"
  # Schema and table configuration
  - name: 'POSTGRES_TABLE'
    value: "{POSTGRES_TABLE}"
  - name: 'POSTGRES_CATEGORY_TABLE'
    value: "{POSTGRES_CATEGORY_TABLE}"
  - name: 'POSTGRES_WAREHOUSE_TABLE'
    value: "{POSTGRES_WAREHOUSE_TABLE}"
  - name: 'POSTGRES_SUPPLIER_TABLE'
    value: "{POSTGRES_SUPPLIER_TABLE}"
  - name: 'POSTGRES_DEMAND_TABLE'
    value: "{POSTGRES_DEMAND_TABLE}"
  - name: 'FORCE_DATA_RESET'
    value: "{FORCE_DATA_RESET}"
  - name: 'LOAD_SAMPLE_DATA'
    value: "{LOAD_SAMPLE_DATA}"
  - name: 'DASHBOARD_ID'
    value: "{DASHBOARD_ID}"
"""

# Write app.yaml to the repository path
app_yaml_path = f"{REPO_WORKSPACE_PATH}/app.yaml"

try:
    # Import workspace API
    from databricks.sdk.service.workspace import ImportFormat
    
    # Write the file using workspace API
    w.workspace.upload(
        path=app_yaml_path,
        content=app_yaml_content.encode('utf-8'),
        format=ImportFormat.AUTO,
        overwrite=True
    )
    
    print(f"✓ app.yaml created successfully!")
    print(f"  Path: {app_yaml_path}")
    print(f"\nConfiguration:")
    print(f"  Database: {PGDATABASE}")
    print(f"  Schema: {POSTGRES_SCHEMA}")
    print(f"  Table: {POSTGRES_TABLE}")
    print(f"  Load Sample Data: {LOAD_SAMPLE_DATA}")
    
except Exception as e:
    print(f"✗ Error creating app.yaml: {e}")
    print(f"\napp.yaml content (copy manually if needed):\n")
    print(app_yaml_content)
    raise

print(f"\n" + "="*60)


## Step 6: Create Databricks App

This step creates the Databricks App or uses an existing one if it already exists.


In [ ]:
print("Step 6: Creating Databricks App...\n")

try:
    # Check if app already exists first
    try:
        app = w.apps.get(name=APP_NAME)
        print(f"⚠ App '{APP_NAME}' already exists. Using existing app.")
        print(f"  App Name: {app.name}")
        if hasattr(app, 'status'):
            print(f"  App Status: {app.status}")
    except Exception as get_error:
        # App doesn't exist, create it
        print(f"App does not exist. Creating new app '{APP_NAME}'...")
        from databricks.sdk.service.apps import App
        
        app_config = App(
            name=APP_NAME,
            description=APP_DESCRIPTION
        )
        
        # Create the app and wait for it to be active
        app = w.apps.create_and_wait(
            app=app_config,
            timeout=timedelta(minutes=20)
        )
        
        print(f"✓ Databricks App '{APP_NAME}' created successfully!")
        print(f"  App Name: {app.name}")
        if hasattr(app, 'status'):
            print(f"  App Status: {app.status}")
    
except Exception as e:
    print(f"✗ Error with app creation: {e}")
    print(f"  Note: The app will be created automatically during deployment in Step 8")

print(f"\n" + "="*60)


## Step 7: Verify App Resources Configuration

App resources (database and secret scope) are now configured in the app.yaml file.


In [ ]:
print("Step 7: Verifying App Resources Configuration...\n")

print("✓ App resources are configured in app.yaml:")
print(f"  - Database: {LAKEBASE_DB_NAME} (Permission: CAN_CONNECT)")
print(f"  - Secret Scope: {SECRET_SCOPE_NAME} (Permission: CAN_READ)")
print()
print("These resources will be automatically attached to the app during deployment.")
print()
print("📝 Note: If automatic resource attachment fails, you may need to add them manually:")
print(f"   1. Go to: {workspace_url}/apps/{APP_NAME}")
print(f"   2. Click 'Edit'->Configure and add 'Resources'")
print(f"   3. Verify Database Resource: {LAKEBASE_DB_NAME}")
print(f"   4. Verify Secret Scope Resource: {SECRET_SCOPE_NAME}")

print(f"\n" + "="*60)


## Step 8: Deploy the Databricks App


In [ ]:
print("Step 8: Deploying Databricks App...\n")

try:
    # Deploy the app
    from databricks.sdk.service.apps import AppDeployment
    
    # Create the AppDeployment object with source code path
    app_deployment = AppDeployment(
        source_code_path=REPO_WORKSPACE_PATH
    )
    
    # Deploy and wait for completion
    deployment = w.apps.deploy_and_wait(
        app_name=APP_NAME,
        app_deployment=app_deployment,
        timeout=timedelta(minutes=20)
    )
    
    print(f"✓ App deployment completed!")
    print(f"  Deployment ID: {deployment.deployment_id}")
    print(f"  Status: {deployment.status}")
    
    # Get the app to check its URL
    app_status = w.apps.get(name=APP_NAME)
    if hasattr(app_status, 'url') and app_status.url:
        print(f"  App URL: {app_status.url}")
    
except Exception as e:
    print(f"✗ Error deploying app: {e}")
    print(f"\nYou can deploy manually using the redeployment command in the cells below.")

print(f"\n" + "="*60)
